In [6]:
%pip install --quiet bs4 newspaper3k lxml_html_clean python-dotenv pymysql mysql-connector-python sqlalchemy

%pip install langchain_text_splitters langchain-community langchain langchain-chroma langchain-teddynote

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install --upgrade newspaper3k

In [7]:
import bs4
from langchain import hub
from langchain_community.llms import GPT4All
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from newspaper import Article
from bs4 import BeautifulSoup
from langchain.document_loaders import WebBaseLoader
from langchain.schema import Document
from sqlalchemy import create_engine, MetaData, Table, select, Column, Integer, String, Text
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from dotenv import load_dotenv
import os
import sys
import requests
import datetime
import time
import json


In [8]:
load_dotenv()

CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")
DB_HOST = os.getenv("DB_HOST")
DB_USERNAME = os.getenv("DB_USERNAME")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_SCHEME = os.getenv("DB_SCHEME")

In [9]:

def search_naver_news(query, display=5, start=1, sort='sim'):
    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET
    }
    params = {
        "query": query,  # 검색어
        "display": display,  # 가져올 결과 수
        "start": start,  # 검색 시작 위치
        "sort": sort  # 정렬 기준: date(날짜순), sim(유사도순)
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json() 
    else:
        print("Error:", response.status_code)
        return None

In [10]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
RAG


In [11]:
engine = create_engine(f'mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}/{DB_SCHEME}')

Session = sessionmaker(bind=engine)
session = Session()

metadata = MetaData()

stock_table = Table('STOCK', metadata, autoload_with=engine)

stmt = select(stock_table.c.STOCK_ID, stock_table.c.STOCK_NAME)

result = session.execute(stmt)

stock_list = [(row[0], row[1]) for row in result]


In [17]:
Base = declarative_base()

class News(Base):
    __tablename__ = 'NEWS'
    news_id = Column(Integer, primary_key=True, autoincrement=True)
    stock_id = Column(Integer, nullable=False)
    title = Column(String(20), nullable=True)
    content = Column(Text, nullable=True)
    link = Column(String(100), nullable=True)
    image = Column(String(100), nullable=True)

engine = create_engine(f'mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}/{DB_SCHEME}')

Session = sessionmaker(bind=engine)
session = Session()

metadata = MetaData()

def save_to_database(news_data):
    try:
        for news_item in news_data:
            data = News(
                stock_id=news_item['stock_id'],
                title=news_item['title'],
                link=news_item['link'],
                content=news_item['content'],
                image=news_item['image']
            )
            session.add(data)

        session.commit()
    except Exception as e:
        session.rollback() 
        print(f"An error occurred: {e}")


C:\Users\David\AppData\Local\Temp\ipykernel_39800\797968463.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [44]:

def parse_news_data(news_data_list, docs, result_list):
  print("start")
  for news_item in news_data_list:
    stock_id = news_item['stock_id']
    for news_url in news_item['news_url_list']:
      if (news_url.startswith("https://n.news.naver.com") == False):
        continue
      response = requests.get(news_url)
      html_content = response.content

      soup = BeautifulSoup(html_content, 'html.parser')

      title = soup.find("div", class_="media_end_head_title").get_text(strip=True)
      content = soup.find("div", class_="newsct_article _article_body").get_text(strip=True)
      # publish_date = soup.find("span", class_="media_end_head_info_datestamp_time").get_text(strip=True)
      image = soup.find("img")['data-src']
      # publisher = soup.find("img", class_="media_end_head_top_logo_img")['title']

      result_list.append({
        'stock_id': stock_id,
        'title': title,
        'content': content,
        'link': news_url,
        # 'publish_date': publish_date,
        'image': image
      })

      doc_content = title + "\n" + content
      doc = Document(page_content=doc_content)
      docs.append(doc)
    # else:
    #   article = Article(news_url, language = 'ko') 

    #   article.download()
    #   article.parse()
    #   title = article.title
    #   text = article.text
    #   date = article.publish_date
    #   image_url = article.top_image

    #   doc_content = title + "\n" + text
    #   doc = Document(page_content=doc_content)
    #   docs.append(doc)

In [45]:
news_data_list = []

for stock_id, stock_name in stock_list:
  result = search_naver_news(stock_name)

  news_url_list = []
  all_splits = []

  if result:
    for idx, item in enumerate(result['items']):
      news_url_list.append(item['link'])

  news_data_list.append({
    'stock_id' : stock_id,
    'news_url_list' : news_url_list
  })
  
  if(len(news_data_list) >= 50):
    print("now saving...")
    result_list = []
    parse_news_data(news_data_list, all_splits, result_list)
    save_to_database(result_list)
    print(f'{len(result_list)} news saved')
    news_data_list = []


Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
now saving...
start
55 news saved
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
now saving...
start
63 news saved
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
now saving...
start
57 news saved
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
now saving...
start
58 news saved
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
now saving...
start
74 news saved
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error

KeyboardInterrupt: 

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

for news_url in news_data:
    article = Article(news_url, language = 'ko') 

    article.download()
    print(news_url)
    article.parse()
    title = article.title
    text = article.text
    date = article.publish_date
    image_url = article.top_image

    doc_content = title + "\n" + text
    doc = Document(page_content=doc_content)
    docs = [doc] 

    # add_to_verctor_db(text_splitter, docs)
    print(f"URL: {news_url}\n {title}\n {text}\n {date}\n{image_url}\n")

In [ ]:
all_splits = []


for news_url in news_data:
    if news_url.startswith("https://n.news.naver.com"):
        # 각 뉴스 URL에 대해 WebBaseLoader를 설정합니다.
        loader = WebBaseLoader(
            web_paths=(news_url,),
            bs_kwargs=dict(
                parse_only=bs4.SoupStrainer(
                    "div",
                    attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
                )
            ),
        )
        article = Article(news_url, language = 'ko') 

        article.download()
        article.parse()

        image_url = article.top_image
        docs = loader.load()
        print('docs :{docs}\n image_url :{image_url}')
    else:

        article = Article(news_url, language = 'ko') 

        article.download()
        article.parse()
        title = article.title
        text = article.text
        date = article.publish_date
        image_url = article.top_image

        doc_content = title + "\n" + text
        doc = Document(page_content=doc_content)
        docs = [doc] 

    print(f"URL: {news_url} {image_url}")
    
    splits = text_splitter.split_documents(docs)
    all_splits.extend(splits)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())

retriever = vectorstore.as_retriever()


In [18]:
from langchain_core.prompts import PromptTemplate

# prompt = PromptTemplate.from_template(
#     """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
# 검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
# 한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

# #Question: 
# {question} 

# #Context: 
# {context} 

# #Answer:"""
# )

prompt = PromptTemplate.from_template(
    """당신은 최신 금융 동향을 분석하고 제공하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context)을 바탕으로 금융 시장에 대한 동향을 전달하는 것입니다.
검색된 다음 문맥(context)을 사용하여 최신 금융 시장 동향을 요약해 주세요. 만약, 주어진 문맥(context)에서 동향을 찾을 수 없다면, `주어진 정보에서 금융 시장 동향에 대한 정보를 찾을 수 없습니다`라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#문맥(Context): 
{context} 

#금융 시장 동향 요약:"""
)

In [19]:
EEVE_Korean_Instruct = "C:/Users/David/AppData/Local/nomic.ai/GPT4All/EEVE-Korean-Instruct-10.8B-v1.0-Q4_0.gguf"  # 원하는 로컬 파일 경로로 대체하세요
gpt4all_falcon = "C:/Users/David/AppData/Local/nomic.ai/GPT4All/gpt4all-falcon-newbpe-q4_0.gguf"
llama3_8b ="C:/Users/David/AppData/Local/nomic.ai/GPT4All/Meta-Llama-3-8B-Instruct.Q4_0.gguf"


In [20]:
llm = GPT4All(
    model=gpt4all_falcon,
    # backend="gpu",  # GPU 설정
    streaming=True,  # 스트리밍 설정
    callbacks=[StreamingStdOutCallbackHandler()],  # 콜백 설정
)

# 체인 생성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
from langchain_teddynote.messages import stream_response

In [ ]:
answer = rag_chain.stream("금융 시장 동향을 요약해주세요.")
stream_response(answer)

In [ ]:
response = llm("한국어로 답변할 수 있습니다?")

response